This is to import all the modules used

In [1]:

import csv as csv
import numpy as np
import pandas
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold




This is to import both the test and train data

In [2]:

#load in csv file train data
data = pandas.read_csv('train.csv', header = 0)

#load in csv file test data
data_test = pandas.read_csv('test.csv', header = 0)


Exploratory analysis on train data - not to be used when running the model

In [3]:


table = pandas.DataFrame.pivot_table(data, index = ['Sex'], columns = 'Survived', aggfunc = np.size)

print table

         PassengerId        Pclass          Name           Age         SibSp  \
Survived           0      1      0      1      0      1      0      1      0   
Sex                                                                            
female          81.0  233.0   81.0  233.0   81.0  233.0   81.0  233.0   81.0   
male           468.0  109.0  468.0  109.0  468.0  109.0  468.0  109.0  468.0   

                 Parch        Ticket          Fare         Cabin         \
Survived      1      0      1      0      1      0      1      0      1   
Sex                                                                       
female    233.0   81.0  233.0   81.0  233.0   81.0  233.0   81.0  233.0   
male      109.0  468.0  109.0  468.0  109.0  468.0  109.0  468.0  109.0   

         Embarked         
Survived        0      1  
Sex                       
female       81.0  233.0  
male        468.0  109.0  


In [4]:
data.describe()

/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Feature Engineering Part 1:
Change some of the string variables to numerical variables. 

In [6]:
#change the Sex variable to Gender
data['Gender']=4
data['Gender'] = data['Sex'].map( {'female':1, 'male':0}).astype(int)

#Fill in missing ages and call AgeFill
data['AgeFill']=data['Age']
median_ages = np.zeros((2,3))

for i in range(0,2):
	for j in range(0,3):
		median_ages[i,j] = data[(data['Gender'] == i ) & (data['Pclass'] ==j+1)]['Age'].dropna().median()


for i in range(0,2):
	for j in range(0,3):
		data.loc[ (data.Age.isnull() & (data['Gender']==i) & (data['Pclass']==j+1)), 'AgeFill']=median_ages[i,j]

#print data[data['Age'].isnull()][['Gender', 'Pclass', 'AgeFill', 'Age']].head(10)

data['Agemissing']=pandas.isnull(data.Age).astype(int)

data['FamilySize'] = data['Parch'] + data['SibSp']

#data.loc(data.Embarked.isnull(), data['Embarked'])='S'

data['Embarked2'] = data['Embarked'].map( {'C':1, 'Q':2, 'S':3, None:3})

data['Agebuckets'] = 2
data.loc[ (data['AgeFill']<15) ,'Agebuckets']=1
data.loc[ (data['AgeFill']>50) ,'Agebuckets']=3

data['Singlewomen']=0
data.loc[((data['AgeFill']>20)&(data['FamilySize']==0)&(data['Gender']==1)),'Singlewomen']=1

data['menHigh']=0
data.loc[((data['Pclass']==1)&(data['Gender']==0)),'menHigh']=1

data['Singlemen']=0
data.loc[((data['AgeFill']>20)&(data['FamilySize']==0)&(data['Gender']==0)),'Singlemen']=1

data['Fare2']=2
data.loc[(data['Fare']<10),'Fare2']=1
data.loc[(data['Fare']>40),'Fare2']=3
print data.info()

data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 22 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Gender         891 non-null int64
AgeFill        891 non-null float64
Agemissing     891 non-null int64
FamilySize     891 non-null int64
Embarked2      891 non-null int64
Agebuckets     891 non-null int64
Singlewomen    891 non-null int64
menHigh        891 non-null int64
Singlemen      891 non-null int64
Fare2          891 non-null int64
dtypes: float64(3), int64(14), object(5)
memory usage: 153.2+ KB
None


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender,AgeFill,Agemissing,FamilySize,Embarked2,Agebuckets,Singlewomen,menHigh,Singlemen,Fare2
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.352413,29.112424,0.198653,0.904602,2.536476,1.984287,0.120090,0.136925,0.408530,1.820426
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990,13.304424,0.399210,1.613459,0.791503,0.399128,0.325249,0.343961,0.491838,0.736884
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400,0.000000,21.500000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200,0.000000,26.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,2.000000
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000,1.000000,36.000000,0.000000,1.000000,3.000000,2.000000,0.000000,0.000000,1.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,80.000000,1.000000,10.000000,3.000000,3.000000,1.000000,1.000000,1.000000,3.000000


In [7]:
    #create new variables for family name and create a variable that predicts whether the family survived or not 
    #clued in as 1 if anyone from the family died


    #create a Last Name column of data
    #data['LastName']=data['Name'].str.split(',').str.get(0)

    le_name = LabelEncoder()
    #data['LastName'] = le_name.fit_transform(data['LastName'])

    data.describe()

    data['Title']=data['Name'].str.split(',').str.get(1).str.split().str.get(0)
    data.loc[(data['Title']=='Mlle.') ,'Title']="Miss."
    data.loc[(data['Title']=="Mme.") ,'Title']="Mrs."
    data.loc[(data['Title']=="Ms.") ,'Title']="Mrs."

    data['Rare']=data['Title'].isin(["Master.", "Miss.", "Mrs.", "Mr."])

    data.loc[(data['Rare']==False),'Title']="Other"
    data = data.drop(['Rare'], axis = 1)

    data['Mother']=0
    data.loc[((data['Title']=="Mrs.")&(data['Parch']>0)),'Mother']=1

    data['Title']=le_name.fit_transform(data['Title'])

    pandas.crosstab(data['Survived'], data['Title']).transpose()

    #classify survival by family type
    #pandas.crosstab(data['Survived'], data['LastName']).transpose()

    #family_died = np.zeros((2,len(data['LastName'])))

    #for i in range(0,len(data['LastName'])):
     #   family_died[0,i]=data[(data['LastName']==i)]['Survived'].sum()

    #for i in range(0,len(data['LastName'])):
     #   if family_died[0,i]==0:
      #      family_died[1,i]=1

    #print family_died.transpose()[10]
    #pandas.crosstab(data['Survived'], data['LastName']).transpose()

    #data['FamilyDied']=0
    #for i in range(0,len(data['LastName'])):
     #   data.loc[(data['LastName']==i),'FamilyDied']=family_died[1,i]

    #data.info()

Survived,0,1
Title,,
0,17,23
1,55,129
2,436,81
3,26,101
4,15,8


In [ ]:
ggplot(data,aes(x='Age',y='PassengerId',color ='Survived'))+ geom_point() +facet_grid('Sex') +\
    scale_color_brewer(type='diverging', palette=4) +xlab("Age") + ylab("PassengerId") + ggtitle("Age and Survived")

Is survival linked to port of embarkment or class?

Survival was worst for embarked S and for class 3. These were most likely the poor individuals. The women who perished seem to have come from the 'S' embarkment. 

Is survival linked to family size?

It looks like men were travelling alone or those with larger families both perished more. Women who were travelling alone tended to survive more. 

In [ ]:
ggplot(data,aes(x='Pclass',y='PassengerId',color ='Survived'))+ geom_point()  +\
    scale_color_brewer(type='diverging', palette=4) +xlab("Pclass") + ylab("PassengerId") + ggtitle("Survival Rates by Class")

In [ ]:
ggplot(data,aes(x='Embarked2',y='PassengerId',color ='Survived'))+ geom_point()  +\
    scale_color_brewer(type='diverging', palette=4) +xlab("Embarked2") + ylab("PassengerId") + ggtitle("Survival Rates by Embarked")

In [ ]:
ggplot(data,aes(x='Embarked2',y='PassengerId',color ='Survived'))+ geom_point() + facet_grid("Sex")+\
    scale_color_brewer(type='diverging', palette=4) +xlab("Embarked2") + ylab("PassengerId") + ggtitle("Survival Rates by Embarked and Gender")

In [ ]:
ggplot(data,aes(x='FamilySize',y='PassengerId',color ='Survived'))+ geom_point() +facet_grid("Sex")+\
    scale_color_brewer(type='diverging', palette=4) +xlab("FamilySize") + ylab("PassengerId") + ggtitle("Survival Rates by Family Size")

Feature Engineering for the test data set.

In [8]:

data_test['Gender']=4
data_test['Gender'] = data_test['Sex'].map( {'female':1, 'male':0}).astype(int)
data_test['AgeFill']=data_test['Age']

median_fare = np.zeros((2,3))

for i in range(0,2):
	for j in range(0,3):
		median_fare[i,j] = data_test[(data_test['Gender'] == i ) & (data_test['Pclass'] ==j+1)]['Fare'].dropna().median()

for i in range(0,2):
	for j in range(0,3):
		data_test.loc[ (data_test.Fare.isnull() & (data_test['Gender']==i) & (data_test['Pclass']==j+1)), 'Fare']=median_fare[i,j]

for i in range(0,2):
	for j in range(0,3):
		data_test.loc[ (data_test.Age.isnull() & (data_test['Gender']==i) & (data_test['Pclass']==j+1)), 'AgeFill']=median_ages[i,j]

data_test['Agemissing']=pandas.isnull(data_test.Age).astype(int)

data_test['FamilySize'] = data_test['Parch'] + data_test['SibSp']

#data_test.loc(data.Embarked.isnull(), 'Embarked')='S'

data_test['Embarked2'] = data_test['Embarked'].map( {'C':1, 'Q':2, 'S':3, None:3})

data_test['Agebuckets'] = 2
data_test.loc[(data_test['AgeFill']<15),'Agebuckets']=1
data_test.loc[(data_test['AgeFill']>50),'Agebuckets']=3

data_test['Singlewomen']=0
data_test.loc[((data_test['AgeFill']>20)&(data_test['FamilySize']==0)&(data_test['Gender']==1)),'Singlewomen']=1

data_test['menHigh']=0
data_test.loc[((data_test['Pclass']==1)&(data['Gender']==0)),'menHigh']=1

data_test['Singlemen']=0
data_test.loc[((data_test['AgeFill']>20)&(data_test['FamilySize']==0)&(data_test['Gender']==0)),'Singlemen']=1

data_test['Fare2']=2
data_test.loc[(data_test['Fare']<10),'Fare2']=1
data_test.loc[(data_test['Fare']>40),'Fare2']=3
print data_test.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 21 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Gender         418 non-null int64
AgeFill        418 non-null float64
Agemissing     418 non-null int64
FamilySize     418 non-null int64
Embarked2      418 non-null int64
Agebuckets     418 non-null int64
Singlewomen    418 non-null int64
menHigh        418 non-null int64
Singlemen      418 non-null int64
Fare2          418 non-null int64
dtypes: float64(3), int64(13), object(5)
memory usage: 68.6+ KB
None


In [9]:
#additional feature engineering for the test data set

#create new variables for family name and create a variable that predicts whether the family survived or not 
#clued in as 1 if anyone from the family died


#create a Last Name column of data
#data_test['LastName']=data_test['Name'].str.split(',').str.get(0)

#le_name = LabelEncoder()
#data_test['LastName'] = le_name.transform(data_test['LastName'])

#data.describe()
#classify survival by family type
#pandas.crosstab(data['Survived'], data['LastName']).transpose()

data_test['Title']=data_test['Name'].str.split(',').str.get(1).str.split().str.get(0)
data_test.loc[(data_test['Title']=='Mlle.') ,'Title']="Miss."
data_test.loc[(data_test['Title']=="Mme.") ,'Title']="Mrs."
data_test.loc[(data_test['Title']=="Ms.") ,'Title']="Mrs."

data_test['Rare']=data_test['Title'].isin(["Master.", "Miss.", "Mrs.", "Mr."])

data_test.loc[(data_test['Rare']==False),'Title']="Other"

#mother variable
data_test['Mother']=0
data_test.loc[((data_test['Title']=="Mrs.")&(data_test['Parch']>0)),'Mother']=1

data_test = data_test.drop(['Rare'], axis = 1)
data_test['Title']=le_name.fit_transform(data_test['Title'])



data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 23 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Gender         418 non-null int64
AgeFill        418 non-null float64
Agemissing     418 non-null int64
FamilySize     418 non-null int64
Embarked2      418 non-null int64
Agebuckets     418 non-null int64
Singlewomen    418 non-null int64
menHigh        418 non-null int64
Singlemen      418 non-null int64
Fare2          418 non-null int64
Title          418 non-null int64
Mother         418 non-null int64
dtypes: float64(3), int64(15), object(5)
memory usage: 75.2+ KB


Drop unused columns for the model

In [10]:
#dropping variables from data

data = data.drop(['Name', 'Age', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Agemissing','AgeFill','Parch','SibSp', 'Fare'], axis = 1)
data = data.drop(['PassengerId'], axis = 1)

print data.describe()
#data = data.values


print data.describe()
#dropping variables from data_test
data_test = data_test.drop(['Name', 'Age', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Agemissing','AgeFill','Parch','SibSp','Fare'], axis = 1)

print data_test.describe()

list_to_write = data_test['PassengerId']
list_to_write = list_to_write.values 

data_test = data_test.drop(['PassengerId'], axis = 1)

data_test = data_test.values

         Survived      Pclass      Gender  FamilySize   Embarked2  Agebuckets  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     0.383838    2.308642    0.352413    0.904602    2.536476    1.984287   
std      0.486592    0.836071    0.477990    1.613459    0.791503    0.399128   
min      0.000000    1.000000    0.000000    0.000000    1.000000    1.000000   
25%      0.000000    2.000000    0.000000    0.000000    2.000000    2.000000   
50%      0.000000    3.000000    0.000000    0.000000    3.000000    2.000000   
75%      1.000000    3.000000    1.000000    1.000000    3.000000    2.000000   
max      1.000000    3.000000    1.000000   10.000000    3.000000    3.000000   

       Singlewomen     menHigh   Singlemen       Fare2       Title      Mother  
count   891.000000  891.000000  891.000000  891.000000  891.000000  891.000000  
mean      0.120090    0.136925    0.408530    1.820426    1.897868    0.062851  
std       0.325249    0.343

Creating separate validation and train datasets from the train data

1. Basic splitting
2. Through cross validation - SKIP
3. Lasso??

In [11]:
#plain splitting
train_data, validation_data = train_test_split(data, test_size = 0.15)


valid_data2=validation_data

validation_data = validation_data.values
train_data=train_data.values



Model Creation: 
    
    1. RandomForestClassifier
    
    #through cross validation


from sklearn.cross_validation import KFold
kf = KFold(len(data['Gender']), n_folds = 4, shuffle = True)
for train_indices, test_indices in kf:
    train_data = [data[ii] for ii in train_indices]
    validation_data = [data[ii] for ii in test_indices]
    
    #cross validation --  SKIP

from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
kf = KFold(len(data['Gender']), n_folds = 4, shuffle = True)
#for train_indices, test_indices in kf:
 #   train_data = [data[ii] for ii in train_indices]
  # clf.fit(train_data[0::,1::], train_data[0::,0])
    #predori = clf.predict(validation_data[0::,1::])
   # predtrain = clf.predict(train_data[0::,1::])
    
d = cross_val_score(clf, train_data[0::,1::],train_data[0::,0],cv=kf, n_jobs=1)
    
    
   2. 
   #Support Vector Machine
from sklearn import svm
clf = svm.SVC(C=0.8)


3. ridge
#Ridge
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier(alpha = .001)
    
    

In [18]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(min_samples_split = 5, warm_start=True)


In [19]:

clf.fit(train_data[0::,1::], train_data[0::,0])
pred = clf.predict(data_test[0::,0::])
predori = clf.predict(validation_data[0::,1::])
predtrain = clf.predict(train_data[0::,1::])

df = pandas.DataFrame(validation_data)


#a = pandas.crosstab(predori, [valid_data2['Pclass'], valid_data2['Gender']])

#b = pandas.crosstab(valid_data2['Survived'], [valid_data2['Pclass'], valid_data2['Gender']])

#print a 
#print b

In [20]:
#print accuracy and confusion matrix and save test predictions is csv file


final = np.concatenate(([list_to_write], [pred]), axis = 0)


#print final

#print list_to_write
from sklearn.metrics import accuracy_score
acc = accuracy_score(predori, validation_data[0::,0])
print "Validation Accuracy: ", acc

acc2 = accuracy_score(predtrain, train_data[0::,0])
print "Train accuracy: ", acc2

from sklearn.metrics import classification_report
print(classification_report(validation_data[0::,0], predori, labels=None,target_names=None,))
#header = "PassengerId, Survived"
#np.set_printoptions(suppress = True)
np.savetxt("mydata.csv", np.transpose(final), delimiter= ",", fmt = '%.1i',comments='',header = "PassengerId,Survived")



Validation Accuracy:  0.820895522388
Train accuracy:  0.866578599736
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        87
          1       0.76      0.72      0.74        47

avg / total       0.82      0.82      0.82       134



In [ ]:
#print model scores for each variable

print clf.oob_decision_function_